In [0]:
import numpy as np
np.set_printoptions(precision=4)


def skew(v):
    return np.array([[0.0, -v[2], v[1]],
                     [v[2], 0.0, -v[0]],
                     [-v[1], v[0], 0.0]])


class Quaternion(object):
    def __init__(self, q0=0, q1=0, q2=0, q3=0):
        self._data = np.array([q0, q1, q2, q3]).flatten()

    @classmethod
    def from_array(cls, arr):
        return Quaternion(*arr[:4])

    @property
    def left_matrix(self):
        q0, q1, q2, q3 = self._data
        matrix = np.array([[q0, -q1, -q2, -q3],
                           [q1,  q0, -q3,  q2],
                           [q2,  q3,  q0, -q1],
                           [q3, -q2,  q1,  q0]])
        return matrix

    @property
    def right_matrix(self):
        q0, q1, q2, q3 = self._data
        matrix = np.array([[q0, -q1, -q2, -q3],
                           [q1,  q0,  q3, -q2],
                           [q2, -q3,  q0,  q1],
                           [q3,  q2, -q1,  q0]])
        return matrix

    @property
    def s(self):
        return self._data[0]

    @property
    def v(self):
        return self._data[1:]

    @property
    def vskew(self):
        return skew(self.v)

    def __neg__(self):
        return Quaternion(*(self._data * -1.0))

    def __mul__(self, other):
        if isinstance(other, (float, np.float)):
            other = Quaternion(other, 0, 0, 0)
        return Quaternion(*np.dot(self.left_matrix, other._data.reshape(4, 1)))

    def __add__(self, other):
        return Quaternion(*(self._data + other._data))

    def __add__(self, other):
        return Quaternion(*(self._data + other._data))

    @property
    def conj(self):
        return Quaternion(self._data[0], -self._data[1], -self._data[2], -self._data[3])

    def spin(self, other):
        return self * other * self.conj

    def norm(self):
        return np.linalg.norm(self._data)

    def normalized(self):
        self._data /= self.norm()
        return self

    def __repr__(self):
        return '[{}, {}, {}, {}]'.format(*self._data)

    def asarray(self):
        return self._data


In [0]:


def recover(v):
    v = v.flatten()
    s = np.sqrt(1 - np.linalg.norm(v[:3])**2)
    ds = - np.inner(v[:3], v[3:]) / s
    dq = DualQuaternion(Quaternion(
        s, v[0], v[1], v[2]), Quaternion(ds, v[3], v[4], v[5]))
    return dq


class DualQuaternion(object):
    def __init__(self, real=Quaternion(), dual=Quaternion()):
        self._real = real
        self._dual = dual

    @classmethod
    def from_array(cls, arr):
        return cls(Quaternion.from_array(arr[:4]),
                   Quaternion.from_array(arr[4:]))

    @property
    def real(self):
        return self._real

    @property
    def dual(self):
        return self._dual

    @property
    def left_matrix(self):
        m8x8 = np.zeros((8, 8))
        m8x8[:4, :4] = self.real.left_matrix
        m8x8[4:, :4] = self.dual.left_matrix
        m8x8[4:, 4:] = self.real.left_matrix
        return m8x8

    @property
    def right_matrix(self):
        m8x8 = np.zeros((8, 8))
        m8x8[:4, :4] = self.real.right_matrix
        m8x8[4:, :4] = self.dual.right_matrix
        m8x8[4:, 4:] = self.real.right_matrix
        return m8x8

    @property
    def s(self):
        return np.array([self.real.s, self.dual.s]).reshape(2, 1)

    @property
    def v(self):
        return np.array([self.real.v, self.dual.v]).reshape(6, 1)

    def inverse(self):
        r = self.real
        d = self.dual
        nsq = (r.conj * r).s
        return DualQuaternion(r.conj * (1.0 / nsq),
                              r.conj * d * r.conj * (-1.0 / nsq**2))

    def __invert__(self):
        return self.inverse()

    @property
    def vskew(self):
        m6x6 = np.zeros((6, 6))
        m6x6[:3, :3] = self.real.vskew
        m6x6[3:, :3] = self.dual.vskew
        m6x6[3:, 3:] = self.real.vskew
        return m6x6

    def __repr__(self):
        return 'DualQuaternion:\n\t''real: ' + self._real.__repr__() +\
            '\n\t' + 'dual: ' + self._dual.__repr__()

    def normalized(self):
        dq = DualQuaternion
        return DualQuaternion(self.real.normalized(),
                              Quaternion(*np.dot(np.eye(4) - np.outer(self.real._data, self.real._data) / self.real.norm()**2,
                                                 self.dual._data.reshape(4, 1))))

    @property
    def conj(self):
        return DualQuaternion(self.real.conj, self.dual.conj)

    def __mul__(self, other):
        if isinstance(other, (float, np.float32)):
            other = DualQuaternion(Quaternion(other, 0, 0, 0), Quaternion())
        return DualQuaternion(self.real * other.real,
                              self.real * other.dual + self.dual * other.real)

    def __add__(self, other):
        return DualQuaternion(self.real + other.real, self.dual + other.dual)

    def __neg__(self):
        return DualQuaternion(-self.real, -self.dual)

    def trs(self):
        return self.dual * self.real.conj * 2.0

    def asarray(self):
        return np.concatenate((self.real.asarray(), self.dual.asarray())).reshape(8, 1)

In [0]:
Q = np.diag([0, 0, 0, 0, 0, 0, 0.0011, 0.0011,
             0.0011, 0.00002, 0.00002, 0.00002]).astype(np.float32)

R = np.diag([0.0000003513, 0.00000259, 0.000003196,
             0.00000547, 0.00000498, 0.0001018]).astype(np.float32)

G = np.eye(12)
G[:6, :6] *= -0.5

H = np.zeros((6, 12))
H[:6, :6] = np.eye(6).astype(np.float32)


def get_F(w):
    F = np.zeros((12, 12))
    F[:6, :6] = -w
    F[:6, 6:] = np.eye(6) * -0.5
    return F


class State(object):
    def __init__(self, x=None, dx=None):
        self.x = x
        self.dx = dx


def time_propagation(P, X, t):

    dq = X.x.conj * X.dx * -0.5
    q = (X.x + dq * t).normalized()

    F = get_F(-X.dx.vskew)

    dP = F @ P + P @ F.T + G @ Q @ G.T

    P += t * dP

    return State(q, X.dx), P


def measurement_update(meas, P, X):
    K = P @ H.T @ np.linalg.inv(H @ P @ H.T + R)

    err = X.x.conj * meas
    err_red = err.v
    delta_err = K @ err_red
    b = delta_err[6:]
    X.x = X.x * recover(delta_err[:6])
    X.dx = X.dx + DualQuaternion(Quaternion(0, b[0], b[1], b[2]),
                                 Quaternion(0, b[3], b[4], b[5]))
    
    dum = np.eye(12) - K @ H
    P = dum @ P @ dum.T + K @ R @ K.T

    return X, P

In [10]:

import matplotlib.pyplot as plt
'''
from rudolph.quaternion import skew
from rudolph.dual_quaternion import recover
from rudolph import Quaternion as Quat
from rudolph import DualQuaternion as DQuat

from rudolph.dqmekf import (G, time_propagation, State, measurement_update)
'''
X = State()
X.x = DualQuaternion(Quaternion(1, 0, 0, 0), Quaternion(0, 0, 0, 0))
X.dx = DualQuaternion()

P = 10e-9 * np.eye(12)

t = np.loadtxt('../datasets/timestamps.txt')
q_icp = np.loadtxt(
    '../datasets/icp_estimates_withinit_NN10_icp10.txt').reshape(-1, 8)

a = []
b = []
c = []

d = []

filts2 = np.loadtxt('../../DQ_filter_lib/filts.txt')
xyzs = []
for f in filts2:
     xyzs.append(DualQuaternion.from_array(f).trs().v.flatten())

xyzs = np.array(xyzs).reshape(-1,3)

measts = []
filts = []

t_s = t[0]
for x in range(1, 197):
    X, P = time_propagation(P, X, t[x] - t_s)
    q = q_icp[x - 1]
    meas = DualQuaternion.from_array(q)
    X, P = measurement_update(meas, P, X)

    # print(X.x)
    # print(meas)
    # print(DQuat.from_array(filts2[x-1]))
    # print(DQuat(Quat(*filts2[x-1][:4]), Quat(*filts2[x-1][4:8])))
    # print()

    filts.append(X.x.trs().v)
    measts.append(meas.trs().v)

    d.append((DualQuaternion(Quaternion(*filts2[x-1][:4]), Quaternion(*filts2[x-1][4:8])).trs().v.flatten()))


    a.append(X.x.asarray().flatten())
    b.append(q.flatten())
    c.append(filts2[x-1][:8])

    t_s = t[x]

a = np.array(a).reshape(-1, 8)
b = np.array(b).reshape(-1, 8)
c = np.array(c).reshape(-1, 8)
d = np.array(d).reshape(-1,3)

f, axarr = plt.subplots(4, sharex=True)
for i in range(4):
    # axarr[i].plot(q_icp[:, i])
    axarr[i].plot(a[:, i])
    axarr[i].plot(b[:, i])
    axarr[i].plot(c[:, i])

measts = np.array(measts).reshape(-1,3)
filts = np.array(filts).reshape(-1,3)

f1, axarr1 = plt.subplots(3, sharex=True)
for i in range(3):
    axarr1[i].plot(measts[:, i])
    axarr1[i].plot(filts[:, i])
    axarr1[i].plot(d[:,i])

plt.show()


OSError: ignored